In [29]:
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.stats import chi2_contingency, norm
from data_viz import *

In [3]:
data = pd.read_csv("data/animal_center.csv")
data

,animal_id,name,animal_type,sex_intake,sex_outcome,breed,color,date_of_birth,found_location,intake_datetime,outcome_datetime,intake_type,intake_condition,outcome_type,outcome_subtype,age_upon_intake(years),age_upon_outcome(years),duration(days)
0,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research in Austin (TX),2014-03-07 14:26:00,2014-03-08 17:10:00,Public Assist,Normal,Return to Owner,NaN,6.7,6.7,1.0
1,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,8700 Research Blvd in Austin (TX),2014-12-19 10:21:00,2014-12-20 16:35:00,Public Assist,Normal,Return to Owner,NaN,7.4,7.4,1.0
2,A006100,Scamp,Dog,Neutered Male,Neutered Male,Spinone Italiano Mix,Yellow/White,2007-07-09,Colony Creek And Hunters Trace in Austin (TX),2017-12-07 14:07:00,2017-12-07 00:00:00,Stray,Normal,Return to Owner,NaN,10.4,10.4,-1.0
3,A047759,Oreo,Dog,Neutered Male,Neutered Male,Dachshund,Tricolor,2004-04-02,Austin (TX),2014-04-02 15:55:00,2014-04-07 15:12:00,Owner Surrender,Normal,Transfer,Partner,10.0,10.0,4.0
4,A134067,Bandit,Dog,Neutered Male,Neutered Male,Shetland Sheepdog,Brown/White,1997-10-16,12034 Research Blvd in Austin (TX),2013-11-16 09:02:00,2013-11-16 11:54:00,Public Assist,Injured,Return to Owner,NaN,16.1,16.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157585,A894036,NaN,Dog,Intact Female,NaN,Miniature Pinscher Mix,Black/Tan,NaN,Martin Luther King Jr Blvd And M Sta Ln in Aus...,2023-12-01 17:33:00,NaN,Stray,Normal,NaN,NaN,NaN,NaN,NaN
157586,A894054,Dawn,Dog,Intact Female,Intact Female,Chihuahua Shorthair,Tan,2022-12-02,2001 Ponciana Loop in Austin (TX),2023-12-02 09:52:00,2023-12-02 13:03:00,Stray,Normal,Rto-Adopt,NaN,1.0,1.0,0.0
157587,A894063,Logan,Cat,Neutered Male,NaN,Domestic Shorthair,Brown Tabby/White,NaN,8585 Scotland Well Drive in Austin (TX),2023-12-02 12:49:00,NaN,Stray,Normal,NaN,NaN,NaN,NaN,NaN
157588,A894067,Maverick,Dog,Intact Male,NaN,Siberian Husky,Black/White,NaN,124 West Anderson Lane in Austin (TX),2023-12-02 13:53:00,NaN,Abandoned,Normal,NaN,NaN,NaN,NaN,NaN


## **Hypothesis4:**
Does age influence the probability of adoption for animals?

$H0$: There is no significant relationship between animals' age and their likelihood of adoption.\
$H1$: There is a significant relationship between animals' age and their likelihood of adoption.

In [23]:
age_adoptiion_research = data[data["age_upon_outcome(years)"] >= 0][["animal_id", "outcome_type", "age_upon_outcome(years)"]].reset_index(drop=True)
age_adoptiion_research

,animal_id,outcome_type,age_upon_outcome(years)
0,A006100,Return to Owner,6.7
1,A006100,Return to Owner,7.4
2,A006100,Return to Owner,10.4
3,A047759,Transfer,10.0
4,A134067,Return to Owner,16.1
...,...,...,...
156563,A894009,Transfer,0.0
156564,A894010,Transfer,0.0
156565,A894011,Transfer,0.0
156566,A894012,Transfer,0.0


In [24]:
age_adoptiion_research.loc[:, "age_group"] = pd.cut(age_adoptiion_research["age_upon_outcome(years)"],
                                                    bins=[-0.01, 2, 7, float('inf')],
                                                    labels=["Young", "Adult", "Senior"])

age_adoptiion_research

,animal_id,outcome_type,age_upon_outcome(years),age_group
0,A006100,Return to Owner,6.7,Adult
1,A006100,Return to Owner,7.4,Senior
2,A006100,Return to Owner,10.4,Senior
3,A047759,Transfer,10.0,Senior
4,A134067,Return to Owner,16.1,Senior
...,...,...,...,...
156563,A894009,Transfer,0.0,Young
156564,A894010,Transfer,0.0,Young
156565,A894011,Transfer,0.0,Young
156566,A894012,Transfer,0.0,Young


In [25]:
age_adoptiion_research["adoption_status"] = np.where(age_adoptiion_research["outcome_type"] == "Adoption", 1, 0)
age_adoptiion_research

,animal_id,outcome_type,age_upon_outcome(years),age_group,adoption_status
0,A006100,Return to Owner,6.7,Adult,0
1,A006100,Return to Owner,7.4,Senior,0
2,A006100,Return to Owner,10.4,Senior,0
3,A047759,Transfer,10.0,Senior,0
4,A134067,Return to Owner,16.1,Senior,0
...,...,...,...,...,...
156563,A894009,Transfer,0.0,Young,0
156564,A894010,Transfer,0.0,Young,0
156565,A894011,Transfer,0.0,Young,0
156566,A894012,Transfer,0.0,Young,0


In [26]:
age_adoption_contingency_table = pd.crosstab(age_adoptiion_research["age_group"], age_adoptiion_research["adoption_status"])
age_adoption_contingency_table

adoption_status,0,1
age_group,,
Young,56315,55733
Adult,18393,14215
Senior,8021,3891


In [28]:
age_groups = ["Young", "Adult", "Senior"]

for age_group in age_groups:
    total = age_adoption_contingency_table.loc[age_group].sum()
    adoption_count = age_adoption_contingency_table.loc[age_group, 1]
    adoption_count_rate = round(adoption_count/total, 4)*100.0
    print(f"The adoption rate for {age_group.lower()} dogs/cats is {adoption_count_rate}%")

The adoption rate for young dogs/cats is 49.74%
The adoption rate for adult dogs/cats is 43.59%
The adoption rate for senior dogs/cats is 32.66%


In [30]:
chi2, p, dof, expected = chi2_contingency(age_adoption_contingency_table)
print(f"chi2:    {chi2}")
print(f"p-value: {p}")
print(f"dof:     {dof}")
print(f"expected:\n{expected}")

chi2:    1470.2129660821583
p-value: 0.0
dof:     2
expected:
[[59205.06739564 52842.93260436]
 [17229.74830106 15378.25169894]
 [ 6294.18430331  5617.81569669]]


In [31]:
chi2, p, dof, expected = chi2_contingency(age_adoption_contingency_table)

# set significance level = 0.05
alpha = 0.05
if p < alpha:
    print("There is a significant relationship between age and adoption status.")
else:
    print("There is no significant relationship between age and adoption status.")

There is a significant relationship between age and adoption status.
